In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
def moc(section, use_density):
    # read stremfunction
    path = f"../sf/single_lats/{section}_sf"
    path = path + "_density.pickle" if use_density else path + "_depth.pickle"
    infile = open(path, "rb")
    _psi = pickle.load(infile); infile.close()
    psi = _psi[:, 1, :].mean(axis=1)
    if use_density: psi = psi[::-1]

    start = np.argmax(psi < 0.)
    sign = lambda x: (x > 0)*2 - 1
    end = np.argmax(sign(psi[start:]) != sign(psi[start])) + start

    # read density/depth range for axis ticks
    if use_density:
        path = f"../sf/single_lats/{section}_density_range.pickle" 
    else:
        path = f"../sf/single_lats/depth_range.pickle"
    infile = open(path, "rb")
    d = pickle.load(infile); infile.close()
    d = [round(x, 2) for x in d]
    d = d[::-1]

    d_0_ix = np.argmax(abs(psi[:end]))
    moc = _psi[::-1][d_0_ix, 1, :]
    d_0 = d[d_0_ix]

    outpath = f"single_lats/{section}_moc_density" if use_density else f"single_lats/{section}_moc_depth"
    outfile = open(f"{outpath}.pickle", "wb")
    pickle.dump(moc, outfile); outfile.close()

    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(moc, color="red", linewidth=1)
    ax.set_ylabel("Abyssal MOC Strength [Sv]")
    ax.set_xlabel("Year")
    ax.set_xticks(np.arange(0, 324, 12), np.arange(1992, 2019), rotation=45)
    ax.set_title(f"{section}: d$_0$={str(round(d_0, 2))}")
    plt.tight_layout()
    plot_title = f"plots/{section}_density" if use_density else f"plots/{section}_depth"
    plt.savefig(f"{plot_title}.png", dpi=400)
    plt.close()

In [3]:
sections = ["26N", "30S", "55S", "60S"]

for section in sections:
    for use_density in [True, False]:
        moc(section, use_density)